In [1]:
import threading
import multiprocessing
import numpy as np
import tensorflow as tf
from AtariEnvironment import *
from ACERNetwork import * 
from ReplayBuffer import *
from Worker import * 
import gym as gym
from time import sleep
from time import time
from utils import preprocess

network_t = "cnn"
GAME_NAME="Breakout-v0"
#GAME_NAME = "LunarLander-v2"
TB_DIR = "saves/breakout/R2_Buf5_1e4/"
env=Atari_Environment(gym.make(GAME_NAME))
#env = gym.make(GAME_NAME)
state= env.reset()
training_finished = False
LEARNING_RATE = 1e-4
print(env.action_space)

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Discrete(4)


/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [4]:
def test_method(game_name=GAME_NAME, num_agents = 1, save_path=TB_DIR):
        
    envs=[]
    agents=[]
    processes=[]
    
    for _ in range(num_agents):
        a_env = Atari_Environment(gym.make(game_name), clipreward=False)
        envs.append(a_env)
       


    with tf.Session() as sess:
        coord = tf.train.Coordinator()
        G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_t,lr=LEARNING_RATE,decay=0.99)
        for agent_num in range(num_agents):
            agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_t))
        saver = tf.train.Saver(max_to_keep=5)

        ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
        saver.restore(sess,ckpt.model_checkpoint_path)

        for thread_id in range(num_agents):
            processes.append(threading.Thread(target=test, args=(agents[thread_id],
            envs[thread_id], 10, False)))

        for p in processes:
            p.daemon = True
            p.start()


        for p in processes:
            p.join()


In [5]:
tf.reset_default_graph()
test_method()

INFO:tensorflow:Restoring parameters from saves/breakout/R2_Buf5_1e4/checkpoints/model2374574.cptk


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Run  0 Reward:  47.0
Run  1 Reward:  147.0
Run  2 Reward:  41.0
Run  3 Reward:  129.0
Run  4 Reward:  91.0
Run  5 Reward:  23.0
Run  6 Reward:  75.0
Run  7 Reward:  87.0
Run  8 Reward:  53.0
Run  9 Reward:  62.0
